# Projekt 1 - Hőmérséklet kiolvasása

Első projektünkben megismerkedünk a DS18b20 szenzorral és, hogy hogyan tudunk belőle pythonból információt kinyerni. Ennek első feltétele, hogy a bevezetőben leírt 1-Wire protokolt már aktiváltuk a Raspberry Pi-on. 

A hőmérséklet kiolvasásának a logikája a következő lesz. Pythonból lefuttatjuk a terminál parancsokat, amik lehetővé teszik a számítógép és a thermisztor közti kommunikációt, majd szoftveresen kiolvassuk a ```w1_slave``` fájlból a hőmérsékletet. 

## Mit fogsz készíteni?

Egy DS18b20 szenzorból (és ha kell egy ellenállásból) álló áramkört készítünk, amiből kiolvasva a hőmérséklet értéket azt megjelenítjük a képernyőn néhány másodpercenként. 

## Mit tanulsz meg?

A hőmérő szenzor elkészítésével a következőket tanulod meg:

* Hogyan futtas terminál parancsokat Pythonból.
* Hogyan keress fájlokat és mappákat az operációs rendszeredben Pythonból a ```glob``` csomaggal.
* Hogyan olvassuk ki egy szöveges fájl tartalmát az ```open``` parancs használatával.
* Hogyan nyerjünk ki hasznos információt egy szöveges fájlból string parancsokat használva.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Előkészíteni a kommunikációt a thermisztorral (```modprobe``` parancsok futtatása).
* Definiálni a hőmérsékletet tartalmazó fájl lokációját.
* Kiolvasni a fájl tartalmát az ```open``` parancs segítségével.
* Definiálni egy függvényt ami a fájlból kinyeri a hőmérsékletet.
* Másodpercenként megismételni a hőmérséklet olvasást.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) DS18b20 szenzor: [itt vásárolhatsz](https://www.tme.eu/hu/details/ds18b20+/homerseklet-jelatalakitok/maxim-integrated/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) Ellenállás: [itt vásárolhatsz](https://www.tme.eu/hu/katalog/metal-film-tht-ellenallasok_112313/?s_order=desc&search=ellenallas&s_field=1000011)

## A kapcsolási rajz

<img src="programok/prog01_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Thermisztorok]() című bevezető leírásban található.

1) Kössük össze a Raspberry Pi egyik földelését a szenzor GND jelű lábával (fekete drót).

2) A szenzor Vcc és Signal lábait kössük össze egy 4.7 kOhmos ellenállással (ez a lépés nem szükséges, ha ez az ellenállás már alapból be van építve). 

3) A szenzor Signal lábát (középső láb) kössük össze a Raspberryn a *GPIO04*-es tüskével (kék drót). 

4) A szenzor Vcc lábát kössük a Raspberry 3.3 V-os táp tüskéjére (piros drót). 

## A kód

Nyissunk meg egy új python fájlt és mentsük el pl. ```ds18b20_test.py``` név alatt. A ```gpiozero``` csomagnak nincs beépített objektuma ami általánosan a thermisztorokkat tudná kezelni, így minden egyes thermisztorhoz nekünk kell megoldani a szoftveres kommunikációt az adott eszközzel.

### A DS18b20 hőmérséklet fájljának előkészítése

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a hőmérséklet kiolvasására.

Mint tudjuk a hőmérséklet értékeket a Raspberry egy kernel parancs elindításával fogja majd a *w1_slave* nevű fájlban tárolni. Első lépésként így be kell olvasnunk azokat a csomagokat, amik lehetővé teszik, hogy parancsokat hajtsunk végre a terminálban. Ezért beimportáljuk az ```os```, ```glob``` és ```time```  csomagokat. Az első segít, hogy parancsokat futtassunk az operációs rendszerben, a második pedig, hogy a dájlrendszerben keresgessünk fájlokat.

```ds18b20_test.py```:

In [1]:
import os
import glob
import time

Le kell futtatnunk a ```modprobe w1-gpio``` és a ```modprobe w1-therm``` parancsokat a terminálból, amire a python ```os.system``` parancsát fogjuk használni. Evvel a python paranccsal tudunk a terminálban bármilyen parancsot lefuttatni. 

```ds18b20_test.py```:

In [ ]:
import os
import glob
import time
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')

Miután a fenti parancsok lefutottak, a Raspberry Pi operációs rendszerében a ```/sys/bus/w1/devices/``` útvonalon meg kell jelenjen a szenzorunk mappája. Ennek a szenzornak az azonosítója, 28-cal kezdődik, így szeretnénk a 28-cal kezdődőelemeket elmenteni egy python változóba. Erre használjuk a ```glob``` csomagot. A ```glob.glob``` metódusának argumentumába írt stringre fog rákeresni a megadott mappában. Itt lehet használni a unix rendszerben használatos wildcard szimbólumokat.

Nézzünk néhány keresési példát. A ```*``` wildcard bármilyen hosszúságú és bármilyen karakterű string egyezést keres. Tehát, ha listázni szeretnénk, hogy a jelenlegi mappánkban mi található, akkor argumentumként a ```'*'```-t adjuk be.

In [2]:
glob.glob('*')

['bevezeto', 'prog01_temperature_onscreen.ipynb', 'Thermistor_bevezeto.ipynb']

Ha arra vagyunk kíváncsiak, hogy egy szintel feljebb a mappákban milyen fájlok/mappák vannak amik *p*-vel kezdődnek akkor:

In [5]:
glob.glob('../p*')

['..\\prog01_temperature_humidity_onscreen.py',
 '..\\prog02_temperature_save_to_file.py',
 '..\\prog031_temperature_continuous_plot.py',
 '..\\prog03_temperature_plot.py',
 '..\\prog041_temperature_led_with_datareading.py',
 '..\\prog04_temperature_led.py',
 '..\\prog05_temperature_button.py',
 '..\\prog06_temperature_motor_fan.py',
 '..\\prog07_temperature_motor_fan_webpage.py']

A mi konkrét esetünkben egy olyan mappát keresünk aminek a neve *28*-cal kezdődik. Ezt a ```glob.glob(base_dir + '28*')``` paranccsal tudjuk listázni. A válasz (feltehetőleg) egy egy elemű lista lesz, így, hogy magát a stringet kapjuk vissza, hivatkozni kell a lista 0-ik elemére, ```device_folder = glob.glob(base_dir + '28*')[0]```. Most már elmenthetjük egy változóba annak a fájlnak a pontos útvonal meghatározását, amiből a hőmérsékleteket szeretnénk majd kiolvasni, ```device_file = device_folder + '/w1_slave'```.

```ds18b20_test.py```:

In [ ]:
import os
import glob
import time
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'

A ```device_file``` lesz a bemenő paraméter az ```open``` parancshoz, ami ki tudja majd olvasni a ```w1_slave``` fájl tartalmát. 

### A w1_slave fájl tartalmának beolvasása

Egy függvényt írunk majd a fájl tartalmának kiolvasására, de ahhoz előbb meg kell ismerkednünk az ```open``` paranccsal.

#### Az open parancs

Az ```open``` parancs szolgál arra, hogy a fájlrendszerben elmentett fájlok tartalmát be tudjuk olvasni a pythonba vagy esetleg pythonból töltsük meg egy fájlt tartalommal. Használata egyszerű, meg kell adni a fájl helyét és nevét a fájlrendzserben és meg kell adni, hogy már létező fájlt szeretnénk-e beolvasni (```r```), szeretnénk-e létrehozni egyet és megtölteni tartalommal (```w```) vagy esetleg a már létező fájl tartalmát szeretnénk kibővíteni (```a```).

In [ ]:
f = open('filename', 'r') # beolvasásra megnyit egy fájlt
f = open('filename', 'w') # új fájl létrehozása
f = open('filename', 'a') # egy létező fájl tartalmának kibővítése

Hozzunk létre először egy fájlt és adjunk neki tartalmat a ```write``` metódussal. 

In [17]:
file = open('testfile.txt','w') 
 
file.write('Hello World\n') 
file.write('Felkészultel a homerseklet kiolvasasara?\n') 
file.write('Lehet tobb sort is be kellene olvasni.\n') 
file.write('Jo buli lesz.') 
 
file.close() 

Itt létrehoztunk egy ```testfile.txt``` nevű fájlt az aktuális mappánkban. A ```write``` metódussal megtöltöttük a fájlt tartalommal. Minden egyes ```write``` parancs ott folytattja a fájl tartalmát ahol az előző abbahagyta. Ha azt szeretnénk, hogy a szövegek új sorba kerüljenek, akkor azt a ```\n``` karakterekkel jelezhetjük. Végül fontos, hogy a kommunikációs csatornát, azaz magát a fájlt is lezárjuk, ```file.close()```, különben a fájlrendszer azt hiszi, hogy még mindig fel szeretnénk tölteni tartalommal és nem lehetne rajta műveleteket végezni. **FONTOS: mindig zárjuk be az open-nel megnyitott kommunikációs csatornákat, ha végeztünk**. 

Most olvassuk be ennek a fáljnak a tartalmát és tároljuk el egy változóban.

In [44]:
file = open('testfile.txt','r')
file.readline()

'Hello World\n'

Amint láthatjuk, a ```readline``` metódus csak egy sort olvasott be. Ha a teljes fájl be akarjuk olvasni akkor a ```read``` metódust kell használni (ügyeljünk arra, hogy ebben az esetben maga a fájl mérete nem nagy (Giga Byte), különben nagyon megterheli a számítógépet). 

In [45]:
file.read()

'Felkészultel a homerseklet kiolvasasara?\nLehet tobb sort is be kellene olvasni.\nJo buli lesz.'

Mivel egy sort már beolvastunk a ```readline``` metódussal, a ```read``` csak a maradék részt olvassa be, ugyanis az olvasás pozíciója elmozdult a fájl elejéről, 0 pozíció. Ha szeretnénk a fájl elejére visszatenni az olvasási pozíciót akkor a ```seek``` metódust kell használnunk és bemenő paraméterként a 0 pozíciót kell megadni.

In [48]:
file.seek(0)

0

A ```readlines``` parancs listába tárolja a beolvasott teljes tartalmat, ahol a lista elemei a beolvasott sorok.

In [49]:
file.readlines()

['Hello World\n',
 'Felkészultel a homerseklet kiolvasasara?\n',
 'Lehet tobb sort is be kellene olvasni.\n',
 'Jo buli lesz.']

In [50]:
file.close()

Ha befejeztük a fájl olvasását, ismét zárjuk be a kommunikációs csatornát a ```close``` metódussal. 

#### Függvény a fájl beolvasására

Ezek után már egyszerű írni egy függvényt, ami kiolvassa a a bemenő paraméterként megadott fájl tartalmát:

In [51]:
def read_temp_raw(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    return lines

In [52]:
read_temp_raw('testfile.txt')

['Hello World\n',
 'Felkészultel a homerseklet kiolvasasara?\n',
 'Lehet tobb sort is be kellene olvasni.\n',
 'Jo buli lesz.']

### A hőmérséklet kinyerése a beolvasott fájlból

Miután képesek vagyunk beolvasni a fájl tartalmát egy python változóba, ki kell nyernünk a hőmérsékletet az ott tárolt információkból. A kiolvasott fájl kb. ezt a szerkezetet fogja felvenni:

In [64]:
txt=['31 00 4b 46 ff ff 05 10 1c : crc=1c YES\n','31 00 4b 46 ff ff 05 10 1c t=24437']

In [65]:
print(txt)

['31 00 4b 46 ff ff 05 10 1c : crc=1c YES\n', '31 00 4b 46 ff ff 05 10 1c t=24437']


Ebből a sok adatból két információra lesz szükségünk. Az első az a lista első elemének a végén lévő válasz (a fenti példában *YES*). Ha a válasz *YES* akkor az adat kiolvasása sikeres volt, ha *NO*, akkor sikertelen. A másik fontos információ az maga a hőmérséklet, amit a lista második elemének *t=* kifejezése után következik. Ezt 1000-el elosztva kapjuk meg a hőmérsékletet Celsius fokban kifejezve. Most nézzük át milyen string műveletekkel nyerhetjük ki ezeket az infokat.

#### String műveletek

Pythonban a stringek karakterekből épülnek fel, amiket indexeléssel ki is nyerhetünk (akár egyszerre többet is):

In [70]:
tt = txt[0]
print(tt[0], tt[3], tt[10])

3 0 6


Ha a 4-ik karaktertől a 20-ik karakterig akarjuk megjeleníteni a szöveget akkor:

In [71]:
print(tt[3:20])

00 4b 46 ff ff 05


Ha hátulról akarunk karaktereket megjeleníteni akkor használhatjuk a negatív számos indexelést:

In [74]:
print(tt[-10:-2])

rc=1c YE


Alapjáraton a *YES* választ többféleképp is megkaphatjuk. Pl. megnézzük, hogy a string tartalmaz-e *YES* karaktert, ```'YES' in txt[0]``` vagy ```txt[0].find('YES')```, stb. Mi most azt vizsgáljuk meg, hogy a string utolsó 3 karaktere megegyezik-e a *YES* kifejezéssel.

In [68]:
txt[0][-3:]

'ES\n'

Amint látjuk, az utolsó karakterekbe beleszámít az is, hogy ott van az újsor parancs. Sőt, ha üres hely lenne, az is egy karakterként számolódna. Így először, el kell tüntetnünk a string sor végére nem kívánatos újsort, üreshelyet stb. Ezt a ```strip``` metódus végzi el a stringen.

In [69]:
txt[0].strip()[-3:]

'YES'

Így már helyesen megkapjuk a kívánt *YES* értéket.

A hőmérséklet megtalálásához arra támaszkodunk, hogy azt mindig megelőzi a *t=* karakterpár. Ha megtaláljuk a stringben a *t=* karakterpár pozícióját, akkor elég csak kiolvasnunk az utána következő összes karaktert és meg lesz a hőmérsékletünk. Ezt a ```find``` metódussal tesszük meg, ahol bemenő paraméterként megadjuk, hogy milyen kifejezést szeretnénk megtalálni a stringben.

In [62]:
pos = txt[1].find('t=')
print(pos)

27


A fenti példában keresett kifejezés a 27-ik pozíciónál jelenik meg. Hozzáadva még két helyi értéket (a *t=* karakterhelyét), megkaphatjuk a hőmérséklet kezdőpozícióját és kiolvashatjuk a teljes hőmérsékletet:

In [63]:
txt[1][pos+2:]

'24437'

#### Függvény a hőmérséklet kinyerésére

Létrehzunk egy függvényt ```read_temp(file_name)```, aminek a bemenő paramétere a hőmérsékleteket tartalmazó fájl neve és ami beolvassa a fájl tartalmát majd kinyeri belőle a hőmérsékletet.

Első lépésként meghívjuk az előző fejezetben definiált függvényt és beolvassuk a fájl tartalmát. A következő lépésben elindítunk egy ```while``` ciklust ami ellenőrzi, hogy a fájl tartalma szerint a hőmérséklet kiolvasás sikeres volt-e, azaz, hogy az első sor 3 utolsó karaktere megegyezik-e a *YES* szóval. Ha megegyezik, akkor kilépünk azonnal a ```while``` ciklusból, de ha nem egyezik, akkor 0.2 másodpercenként újra kiolvassuk a fájl tartalmát, addig amíg a hőmérséklet olvasás sikeres nem lesz.

Sikeres hőmérséklet kiolvasás esetén a fájl második sorában megkeressük a *t=* karakterpárnak a helyét. Ha ez a karakterpár nincs abban a sorban, akkor a ```find('t=')``` metódus válasza -1 lesz és ebben az esetben semmit nem teszünk. Ha mégis sikerül megtalálni a karakterpár pozícióját, akkor kiolvassuk a karakterpár után következő összes karaktert, ```temp_string = lines[1][equals_pos+2:]```. Mivel ennek a típusa string, így számmá kell alakítanunk a ```float``` paranccsal, hogy aritmetikai műveleteket végezhessünk rajta, azaz 1000-el elosztva átalakíthassuk Celsius fokká, ```temp_c = float(temp_string) / 1000.0```. Ha ismerjük a hőmérsékletet Celsius fokban akkor a $t_F = t_C \times \frac{9}{5} + 32$ képlettel átalakítjuk Fahrenheitba is, majd mindkét értéket visszaadjuk.

In [ ]:
def read_temp(file_name):
    lines = read_temp_raw(file_name)
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw(file_name)
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        temp_f = temp_c * 9.0 / 5.0 + 32.0
        return temp_c, temp_f

### A hőmérséklet megjelenítése a képernyőn

Nem maradt más hátra, mint a fent leírt programrészleteket egybegyúrni. Végül meghívunk egy végtelen ```while``` ciklust, hogy a hőmérsékletet újra és újra kiolvassuk és kiírjuk a képernyőre. A kiolvasások közé 1 másodperc szünetet teszünk.

```ds18b20_test.py```:

In [ ]:
import os
import glob
import time
 
os.system('modprobe w1-gpio')
os.system('modprobe w1-therm')
 
base_dir = '/sys/bus/w1/devices/'
device_folder = glob.glob(base_dir + '28*')[0]
device_file = device_folder + '/w1_slave'
 
def read_temp_raw(file_name):
    f = open(file_name, 'r')
    lines = f.readlines()
    f.close()
    return lines
 
def read_temp(file_name):
    lines = read_temp_raw(file_name)
    while lines[0].strip()[-3:] != 'YES':
        time.sleep(0.2)
        lines = read_temp_raw(file_name)
    equals_pos = lines[1].find('t=')
    if equals_pos != -1:
        temp_string = lines[1][equals_pos+2:]
        temp_c = float(temp_string) / 1000.0
        temp_f = temp_c * 9.0 / 5.0 + 32.0
        return temp_c, temp_f
	
while True:
	print(read_temp(device_file))	
	time.sleep(1)

Ha a program működik és fut, a ```CTRL+C``` billentyűpárral tudjuk leállítani.

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```ds18b20_test.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```ds18b20_test.py```-t elmentettük. Ott begépelve a ```python ds18b20_test.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a program elindításával a képernyőre kiíródik az aktuális hőmérséklet celsius és fahrenheit fokban is.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* Módosítsuk a kódot úgy, hogy ne csak a hőmérsékletet írja ki, hanem az hőmérséklet leolvasásának időpontját is.
* Ha a hőmérséklet 30 Celsius fok fölé emelkedne, nyomtassuk ki a képernyőre, hogy Kánikula (és ne írassuk ki a hőmérsékletet) ameddig vissza nem csökken 30 Celsius fok alá.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) https://thepihut.com/blogs/raspberry-pi-tutorials/18095732-sensors-temperature-with-the-1-wire-interface-and-the-ds18b20

2) http://docs.37sensors.com/#

3) https://sensorkit.en.joy-it.net/index.php?title=KY-028_Temperature_Sensor_module_(Thermistor)

4) https://www.malnasuli.hu/leckek/homerseklet-merese-1-wire-szenzorral/

5) https://www.electronicshub.org/raspberry-pi-ds18b20-tutorial/

6) open parancs - https://www.guru99.com/reading-and-writing-files-in-python.html

7) glob csomag - https://pymotw.com/2/glob/

8) os csomag - https://www.python-course.eu/os_module_shell.php

9) string metódusok - https://www.programiz.com/python-programming/string